In [1]:
from bs4 import BeautifulSoup
import dataset
import unicodedata

In [2]:
db = dataset.connect("sqlite:///vdc.sqlite")

In [3]:
db.tables

['content_ar', 'content_en', 'keys_ar', 'keys_en', 'links']

In [4]:
tab_en = db['content_en']
tab_ar = db['content_ar']

In [5]:
# records_ar = []
# records_en = []

# for r in tab_ar.find():
#     records_ar.append(r)
    
# for r in tab_en.find():
#     records_en.append(r)

In [6]:
entries = []

def db_entry_for_record(raw_record):
    thisrec = {"url":raw_record['url'], 'link_id':raw_record['link_id']}
    try:
        soup = BeautifulSoup(raw_record['content'],"lxml")    
        k = None
        v = None

        for idx,elem in enumerate(soup.find_all('td')[2:]):
            if idx % 2 == 0:
                k = unicodedata.normalize("NFKD", elem.text)
                if k == ' ':
                    k = 'video_description'
                if ('\n' in k):
                    thisrec['video_link'] = k
            else:
                v = unicodedata.normalize("NFKD", elem.text)
                if v.strip() == '':
                    v = None
                thisrec[k] = v
    except:
        # None record for English where there was an
        # Arabic page and no English page
        pass
    
    # Remove where youtube links become keys
    for k in list(thisrec.keys()):
        if ('\n' in k):
            del thisrec[k]
    
    return thisrec

In [7]:
# Make a comprehensive list of the keys in the content
# arabic_keys = {}
# english_keys = {}

# for r in tab_ar.find():
#     db_entry = db_entry_for_record(r)
#     if db_entry is not None:
#         for k in db_entry.keys():
#             arabic_keys[k] = ''
            
# for r in tab_en.find():
#     db_entry = db_entry_for_record(r)
#     if db_entry is not None:
#         for k in db_entry.keys():
#             english_keys[k] = '' 
            
# tab_ar_keys = db['keys_ar']
# tab_en_keys = db['keys_en']

# for k in arabic_keys.keys():
#     tab_ar_keys.insert({'orig':k, 'standardized':''})
    
# for k in english_keys.keys():
#     tab_en_keys.insert({'orig':k, 'standardized':''})

In [8]:
# english_keys
tab_en_keys = db['keys_en']

english_keys_data = {}

en_keys = tab_en_keys.find()

for enk in en_keys:
    english_keys_data[enk['orig']] = enk['standardized']
    
english_keys_data

{'Actors': 'actors',
 'Age': 'age',
 'Area \\ Place of birth': 'place_of_birth',
 'Brigade Name': 'faction_name',
 'Caesar File': 'caesar_file',
 'Cause of Death': 'cause_of_death',
 'Date of Birth': 'date_of_birth',
 'Date of death': 'date_of_death',
 'Direct Cause of Death': 'direct_case_of_death',
 'Facebook Page': 'facebook_page',
 'ID Card number': 'id_card_number',
 'Link': 'link',
 'Marital Status': 'marital_status',
 'Martyrdom location': 'place_of_death',
 'Method of Execution': 'method_of_execution',
 "Mother's Name": 'mothers_name',
 'Name': 'name',
 'National Number': 'national_number',
 'News Article': 'news_article',
 'Notes': 'notes',
 'Number of kids': 'number_of_children',
 'Occupation': 'occupation',
 'Photo': 'photo',
 'Province': 'province',
 'Rank / Military situation': 'rank_military_status',
 'Sex': 'sex',
 'Status': 'status',
 'The Source': 'source',
 'Type of Weapon': 'type_of_weapon',
 'link_id': 'link_id',
 'url': 'url',
 'video_description': 'video_descripti

In [9]:
# arabic_keys
tab_ar_keys = db['keys_ar']

arabic_keys_data = {}

ar_keys = tab_ar_keys.find()

for ark in ar_keys:
    arabic_keys_data[ark['orig']] = ark['standardized']
    
arabic_keys_data

{'link_id': 'link_id',
 'url': 'url',
 'video_description': 'video_description',
 'video_link': 'video_link',
 'الاسم': 'name',
 'الجنس': 'sex',
 'الجهة الفاعلة': 'actors',
 'الحالة الزوجية': 'marital_status',
 'الرابط': 'link',
 'الرتبة/الوضع العسكري': 'rank_military_status',
 'الرقم الوطني': 'national_number',
 'السبب المباشر للوفاة': 'direct_cause_of_death',
 'العمر': 'age',
 'الفئة': 'status',
 'المحافظة': 'province',
 'المصدر': 'source',
 'المنطقة / مكان التولد': 'place_of_birth',
 'المهنه': 'occupation',
 'إسم الام': 'mothers_name',
 'إسم الفصيل': 'faction_name',
 'تاريخ التولد': 'date_of_birth',
 'تاريخ الوفاة': 'date_of_death',
 'رقم الهويه': 'id_card_number',
 'سبب الوفاة': 'cause_of_death',
 'صفحة فيسبوك': 'facebook_page',
 'صورة شخصيه': 'photo',
 'طريقة الإعدام': 'method_of_execution',
 'عدد الأولاد': 'number_of_children',
 'معلومات اضافيه': 'notes',
 'مقال صحفي': 'news_article',
 'مكان الوفاة': 'place_of_death',
 'ملف سيزر': 'caesar_file',
 'نوع السلاح': 'type_of_weapon'}

In [10]:
tab_en_final = db['final_english']
            
for r in tab_en.find():
    db_entry = db_entry_for_record(r)
    if db_entry is not None:
        new_rec = {}
        for k in db_entry.keys():
            new_rec[english_keys_data[k]] = db_entry[k]
        tab_en_final.insert(new_rec)

In [11]:
tab_ar_final = db['final_arabic']
            
for r in tab_ar.find():
    db_entry = db_entry_for_record(r)
    if db_entry is not None:
        new_rec = {}
        for k in db_entry.keys():
            new_rec[arabic_keys_data[k]] = db_entry[k]
        tab_ar_final.insert(new_rec)